### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


import copy
import seaborn as sns
import os
from scipy import stats
import datetime

%matplotlib inline

In [2]:
data_path = '/mnt/d/lighthouse/Midterm_data/'

In [21]:
# 15927485 rows × 42
flights = pd.read_csv(data_path + 'flights.csv', header = 0, low_memory=False)
flights_test = pd.read_csv(data_path + 'flights_test.csv', header = 0)
flights_weather = pd.read_csv(data_path + 'wkey.csv', header = 0)
flights_weather.drop('Unnamed: 0', axis = 1, inplace = True)

In [22]:
# making duplicate so we dont have to re-read large csv files
flights_clean = copy.deepcopy(flights)

## Nan

In [23]:
total = flights_clean.isnull().sum().sort_values(ascending = False)
percent = (flights_clean.isnull().sum()/flights_clean.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])

In [24]:
#missing_data.head(15)

In [25]:
# dropping columsn with 80% missing AND [origin_city_name, dest_city_name] since we have [orgin, dest]
# dropping [cancelled, diverted, flights, dup] since they only contain one value
cols_to_drop_nan = list(missing_data[missing_data['Percent'] > 0.8].index)
#ols_to_drop_other = ['dep_delay','taxi_out','taxi_in', 'wheels_off', 'wheels_on', 'arr_time', 'cancelled', 'diverted','actual_elapsed_time','air_time', 'flights', 'dup', 'dep_time']
cols_to_drop_other = ['taxi_out','taxi_in', 'wheels_off', 'wheels_on', 'cancelled', 'diverted','actual_elapsed_time','air_time', 'flights', 'dup', 'dep_time']
flights_clean.drop(cols_to_drop_nan, axis = 1, inplace = True)
flights_clean.drop(cols_to_drop_other, axis = 1, inplace = True)

## rows with Nan

In [26]:
# before: (15927485, 20)
# after: 15652397 rows × 20 columns
# lost 3% of data, no biggie
flights_clean.dropna(inplace = True)

# Merging weather data

In [29]:
flights_weather

,origin,month,latitude,longitude,wdir,period,temp,maxt,visibility,wspd,...,mint,precip,snowdepth,sealevelpressure,dew,humidity,wgust,precipcover,windchill,closest_airport
0,LAX,1,33.942540,-118.408070,155.18,Jan,59.3,87.8,8.7,31.4,...,40.0,6.98,NaN,1018.8,44.4,64.04,40.0,7.40,35.2,LAX
1,SFO,1,37.619000,-122.374840,155.18,Jan,59.3,87.8,8.7,31.4,...,40.0,6.98,NaN,1018.8,44.4,64.04,40.0,7.40,35.2,LAX
2,SAN,1,32.733560,-117.189660,155.18,Jan,59.3,87.8,8.7,31.4,...,40.0,6.98,NaN,1018.8,44.4,64.04,40.0,7.40,35.2,LAX
3,SEA,1,47.448980,-122.309310,175.30,Jan,35.7,59.1,7.9,26.4,...,19.1,2.37,NaN,1020.7,29.9,80.47,44.7,6.59,13.4,YKM
4,RDU,1,35.877640,-78.787470,215.02,Jan,32.8,70.0,9.1,35.8,...,-0.9,5.37,3.6,1021.9,21.7,66.20,50.8,9.01,-10.1,IAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4507,PAE,12,40.969990,-77.727880,197.82,Dec,38.8,65.9,8.2,26.4,...,19.1,8.68,NaN,1019.4,30.4,73.73,49.4,12.84,6.8,IAD
4508,PIR,12,48.871307,-0.734151,194.17,Dec,42.1,58.7,7.9,31.0,...,24.6,7.37,0.0,1016.2,40.0,92.33,51.1,14.78,16.6,FAY
4509,ATY,12,47.660583,50.806202,261.74,Dec,39.9,55.9,9.2,16.4,...,22.2,22.43,0.3,NaN,33.0,78.55,33.3,10.14,26.5,DIK
4510,YNG,12,36.526947,58.215620,261.74,Dec,39.9,55.9,9.2,16.4,...,22.2,22.43,0.3,NaN,33.0,78.55,33.3,10.14,26.5,DIK


In [32]:
flights_clean.drop(['mkt_carrier','op_unique_carrier','op_carrier_fl_num','origin_airport_id','dest_airport_id','dest_city_name', 'origin_city_name','crs_elapsed_time'], axis =1, inplace = True)

In [34]:
flights_clean['fl_date'] = pd.to_datetime(flights_clean['fl_date'])
flights_clean['year'] = flights_clean['fl_date'].dt.year
flights_clean['month'] = flights_clean['fl_date'].dt.month
flights_clean['date'] = flights_clean['fl_date'].dt.day
flights_clean.drop('fl_date', axis = 1, inplace = True)

In [35]:
flights_clean

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,tail_num,origin,dest,crs_dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,distance,year,month,date
0,WN,WN,2098,N8540V,MCI,PHX,1755,-2.0,1850,1838.0,-12.0,1044,2019,3,25
1,WN,WN,2238,N8656B,MCI,PHX,2000,-5.0,2055,2036.0,-19.0,1044,2019,3,25
2,WN,WN,2451,N8583Z,MCI,PHX,540,1.0,635,625.0,-10.0,1044,2019,3,25
3,WN,WN,2213,N737JW,MCI,RDU,1550,99.0,1905,2039.0,94.0,904,2019,3,25
4,WN,WN,2096,N705SW,MCI,RSW,1045,166.0,1430,1702.0,152.0,1155,2019,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15927480,WN,WN,2189,N7702A,MCI,MSY,2055,16.0,2245,2242.0,-3.0,689,2019,3,25
15927481,WN,WN,1291,N7878A,MCI,OAK,1200,88.0,1355,1534.0,99.0,1489,2019,3,25
15927482,WN,WN,2470,N760SW,MCI,PDX,920,-5.0,1110,1052.0,-18.0,1482,2019,3,25
15927483,WN,WN,1651,N8542Z,MCI,PHX,1125,-2.0,1215,1204.0,-11.0,1044,2019,3,25


In [36]:
flights_clean = flights_clean.merge(flights_weather, on=['origin','month'], how='left')

In [38]:
flights_clean.to_csv('/mnt/d/lighthouse/Midterm_data/model_flights_with_origin_weather.csv')

## flights_checkpoint (dropped Nan columns and rows)

In [3]:
# 15611141 rows × 21 columns
flights_clean = pd.read_csv(data_path + 'flights_cleaned_model.csv', header = 0, low_memory=False)
flights_clean.drop('Unnamed: 0', axis = 1, inplace = True)
flights_test =pd.read_csv(data_path + 'flights_test.csv', header = 0)

### seperate year-month-day

In [4]:
flights_clean['fl_date'] = pd.to_datetime(flights_clean['fl_date'])
flights_clean['year'] = flights_clean['fl_date'].dt.year
flights_clean['month'] = flights_clean['fl_date'].dt.month
flights_clean['date'] = flights_clean['fl_date'].dt.day
flights_clean.drop('fl_date', axis = 1, inplace = True)

flights_test['fl_date'] = pd.to_datetime(flights_test['fl_date'])
flights_test['year'] = flights_test['fl_date'].dt.year
flights_test['month'] = flights_test['fl_date'].dt.month
flights_test['date'] = flights_test['fl_date'].dt.day
# Dropping the old date column
flights_test.drop(['fl_date','dup','flights'], axis = 1, inplace = True)

In [5]:
flights_clean

,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,crs_dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,distance,year,month,date
0,WN,WN,WN,2098,WN,N8540V,2098,13198,MCI,"Kansas City, MO",...,1755,-2.0,1850,1838.0,-12.0,175.0,1044,2019,3,25
1,WN,WN,WN,2238,WN,N8656B,2238,13198,MCI,"Kansas City, MO",...,2000,-5.0,2055,2036.0,-19.0,175.0,1044,2019,3,25
2,WN,WN,WN,2451,WN,N8583Z,2451,13198,MCI,"Kansas City, MO",...,540,1.0,635,625.0,-10.0,175.0,1044,2019,3,25
3,WN,WN,WN,2213,WN,N737JW,2213,13198,MCI,"Kansas City, MO",...,1550,99.0,1905,2039.0,94.0,135.0,904,2019,3,25
4,WN,WN,WN,2096,WN,N705SW,2096,13198,MCI,"Kansas City, MO",...,1045,166.0,1430,1702.0,152.0,165.0,1155,2019,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15611136,WN,WN,WN,2189,WN,N7702A,2189,13198,MCI,"Kansas City, MO",...,2055,16.0,2245,2242.0,-3.0,110.0,689,2019,3,25
15611137,WN,WN,WN,1291,WN,N7878A,1291,13198,MCI,"Kansas City, MO",...,1200,88.0,1355,1534.0,99.0,235.0,1489,2019,3,25
15611138,WN,WN,WN,2470,WN,N760SW,2470,13198,MCI,"Kansas City, MO",...,920,-5.0,1110,1052.0,-18.0,230.0,1482,2019,3,25
15611139,WN,WN,WN,1651,WN,N8542Z,1651,13198,MCI,"Kansas City, MO",...,1125,-2.0,1215,1204.0,-11.0,170.0,1044,2019,3,25


In [6]:
#flights_clean[list(flights_test.columns)]

In [7]:
set(flights_test.columns)^set(flights_clean.columns)

{'arr_delay', 'arr_time', 'dep_delay'}

# Feature engineering

### Average delay for tail_num (specific airplane)

In [8]:
flights_FE = copy.deepcopy(flights_clean)
flights_FE = flights_FE[(flights_FE['dep_delay']>0) | (flights_FE['arr_delay']>0)]

In [9]:
delay_df = flights_FE.groupby('tail_num').mean()[['arr_delay','dep_delay']]
delay_df.reset_index(inplace=True)
delay_df.columns = ['tail_num','mean_tail_num_arr_delay','mean_tail_num_dep_delay']
delay_df

,tail_num,mean_tail_num_arr_delay,mean_tail_num_dep_delay
0,215NV,33.043988,31.904203
1,216NV,36.282759,33.558621
2,217NV,43.742410,41.177553
3,218NV,38.836638,36.399433
4,219NV,39.116352,37.131027
...,...,...,...
6473,N999DN,23.850977,25.293160
6474,N999FR,25.375000,18.750000
6475,N999JB,21.244186,22.813953
6476,N999JQ,41.288714,45.611549


In [10]:
flights_clean = flights_clean.merge(delay_df, on=['tail_num'], how='left')

### Average delay by carrier

In [11]:
carrier_delay = flights_FE.groupby('mkt_unique_carrier').mean()[['arr_delay','dep_delay']]
carrier_delay.reset_index(inplace=True)
carrier_delay.columns = ['mkt_unique_carrier','mean_carrier_arr_delay','mean_carrier__dep_delay']
carrier_delay

,mkt_unique_carrier,mean_carrier_arr_delay,mean_carrier__dep_delay
0,AA,31.420120,29.823116
1,AS,21.034051,18.238872
2,B6,40.321600,41.194944
3,DL,30.814713,31.867638
4,F9,37.903671,39.483474
5,G4,37.406556,35.583511
6,HA,15.450311,12.976746
7,NK,33.288398,33.350894
8,UA,41.523520,38.614889
9,VX,24.528221,22.637083


In [12]:
flights_clean = flights_clean.merge(carrier_delay, on=['mkt_unique_carrier'], how='left')

### average delay by mkt_flight_number

In [13]:
fl_num_delay = flights_FE.groupby('mkt_carrier_fl_num').mean()[['arr_delay','dep_delay']]
fl_num_delay.reset_index(inplace=True)
fl_num_delay.columns = ['mkt_carrier_fl_num','mean_fl_num_arr_delay','mean_fl_num_dep_delay']
fl_num_delay

,mkt_carrier_fl_num,mean_fl_num_arr_delay,mean_fl_num_dep_delay
0,1,13.480081,13.680621
1,2,34.126994,36.820245
2,3,26.202833,26.724646
3,4,26.173761,25.998498
4,5,20.870787,22.958567
...,...,...,...
7191,9390,21.000000,18.210526
7192,9391,251.166667,249.916667
7193,9392,246.800000,226.000000
7194,9393,28.500000,35.000000


In [14]:
flights_clean = flights_clean.merge(fl_num_delay, on=['mkt_carrier_fl_num'], how='left')

In [20]:
flights_clean

,branded_code_share,origin_airport_id,dest_airport_id,dep_delay,arr_delay,distance,year,month,date,mean_tail_num_arr_delay,mean_tail_num_dep_delay,mean_carrier_arr_delay,mean_carrier__dep_delay,mean_fl_num_arr_delay,mean_fl_num_dep_delay
0,0,13198,14107,-2.0,-12.0,1044,2019,3,25,12.627563,18.052392,17.308595,22.411967,24.737259,24.268403
1,0,13198,14107,-5.0,-19.0,1044,2019,3,25,14.087955,20.485714,17.308595,22.411967,25.317333,26.811733
2,0,13198,14107,1.0,-10.0,1044,2019,3,25,12.455556,18.637037,17.308595,22.411967,20.055714,24.110000
3,0,13198,14492,99.0,94.0,904,2019,3,25,17.657185,22.203531,17.308595,22.411967,27.390947,29.937449
4,0,13198,14635,166.0,152.0,1155,2019,3,25,19.461314,24.219465,17.308595,22.411967,20.840935,22.476625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15611136,0,13198,13495,16.0,-3.0,689,2019,3,25,20.831000,25.552873,17.308595,22.411967,25.820411,26.954905
15611137,0,13198,13796,88.0,99.0,1489,2019,3,25,19.259850,24.471857,17.308595,22.411967,24.652507,27.632312
15611138,0,13198,14057,-5.0,-18.0,1482,2019,3,25,20.524267,25.563671,17.308595,22.411967,24.448864,23.370455
15611139,0,13198,14107,-2.0,-11.0,1044,2019,3,25,12.019767,18.370930,17.308595,22.411967,26.425791,25.776156


In [16]:
flights_clean.drop(['mkt_unique_carrier', 'mkt_carrier', 'mkt_carrier_fl_num', 'tail_num','op_unique_carrier', 'op_carrier_fl_num','origin','origin_city_name','dest','dest_city_name','crs_dep_time','crs_arr_time','arr_time','crs_elapsed_time'],axis = 1, inplace = True)

In [18]:
# converting categorical to numbers
flights_clean['branded_code_share'] = pd.factorize(flights_clean['branded_code_share'])[0]

In [46]:
#flights_clean.to_csv('/mnt/d/lighthouse/Midterm_data/model_4_FE_mean_delay.csv')

# Model

In [47]:
flights_final = pd.read_csv(data_path + 'model_4_FE_mean_delay.csv', header = 0)
flights_final.drop('Unnamed: 0', axis = 1, inplace= True)

In [59]:
flights_final.drop(['mkt_unique_carrier', 'tail_num'], axis =1, inplace = True)
flights_final.dropna(inplace = True)

In [60]:
flights_final

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,arr_delay,distance,year,month,date,mean_arr_delay,mean_dep_delay,mean_carrier_arr_delay,mean_carrier__dep_delay
0,2098,13198,14107,-12.0,1044,2019,3,25,12.627563,18.052392,17.308595,22.411967
1,2238,13198,14107,-19.0,1044,2019,3,25,14.087955,20.485714,17.308595,22.411967
2,2451,13198,14107,-10.0,1044,2019,3,25,12.455556,18.637037,17.308595,22.411967
3,2213,13198,14492,94.0,904,2019,3,25,17.657185,22.203531,17.308595,22.411967
4,2096,13198,14635,152.0,1155,2019,3,25,19.461314,24.219465,17.308595,22.411967
...,...,...,...,...,...,...,...,...,...,...,...,...
15611136,2189,13198,13495,-3.0,689,2019,3,25,20.831000,25.552873,17.308595,22.411967
15611137,1291,13198,13796,99.0,1489,2019,3,25,19.259850,24.471857,17.308595,22.411967
15611138,2470,13198,14057,-18.0,1482,2019,3,25,20.524267,25.563671,17.308595,22.411967
15611139,1651,13198,14107,-11.0,1044,2019,3,25,12.019767,18.370930,17.308595,22.411967


In [70]:
X = flights_final.drop('arr_delay', axis = 1)
y = flights_final['arr_delay']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

### randomsearch for hyperparameters

In [111]:
one_to_left = stats.beta(10, 1)  
from_zero_positive = stats.expon(0, 50)

params = {  
    "n_estimators": stats.randint(3, 100),
    "max_depth": stats.randint(5, 10),
    "learning_rate": stats.uniform(0.01, 0.5),
    "colsample_bytree": one_to_left,
    "gamma": stats.uniform(0, 10),
    'reg_alpha': from_zero_positive
}

xg_reg = xgb.XGBRegressor(nthreads=-1)  

In [112]:
clf = RandomizedSearchCV(xg_reg, params, n_jobs=1)  
clf.fit(X_train, y_train)  

[23:10:54] WARNING: /workspace/src/learner.cc:480: 
Parameters: { nthreads } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:10:55] WARNING: /workspace/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[23:20:14] WARNING: /workspace/src/learner.cc:480: 
Parameters: { nthreads } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:20:14] WARNING: /workspace/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for 

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs...
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f124e1aa730>,
                                        'learning_rate': <scipy.stats._distn_infrastructu

In [134]:
set(flights['origin'].unique())

376

In [138]:
set(flights['origin'].unique())-set(flights_test['origin'].unique())

{'ACK',
 'AKN',
 'BKG',
 'DLG',
 'DUT',
 'GST',
 'HYA',
 'IFP',
 'ISN',
 'MVY',
 'ROP',
 'WYS',
 'YNG'}

In [139]:
set(flights_test['origin'].unique())-set(flights['origin'].unique())

{'RIW', 'SHR'}

### XGB

In [144]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror',booster='gbtree', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 100)

In [145]:
xg_reg.fit(X_train,y_train)

[12:12:23] WARNING: /workspace/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [146]:
preds = xg_reg.predict(X_test)

In [147]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 49.077391


### RandomForestRegressor

In [149]:
regr = RandomForestRegressor(random_state=0)

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
y_pred = regr.predict(x_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse